In [125]:
#https://www.bbc.com/russian
import ibm_db_dbi as db
import requests
import re
from bs4 import BeautifulSoup
import urllib.request
import urllib
import psycopg2
from psycopg2 import Error
import datetime
from datetime import date, timedelta, datetime

date_while = date.today()
xx = date_while.strftime('%Y-%m-%d')
current_time = datetime.now()

def telegram_bot_sendtext(bot_message):
    bot_token = '1700296384:AAGoaq3o2WD7BbUthmU9dEYQD5IMEKBIgxU'
    bot_chatID = '771779345'
    send_text = 'https://api.telegram.org/bot' + bot_token + '/sendMessage?chat_id=' + bot_chatID + '&parse_mode=Markdown&text=' + bot_message
    response = requests.get(send_text)
    return response.json()

def converter(url):
    url = re.sub(r'[\W+\_]','',url)
    object_id = ''
    for u in url:
        object_id += str(ord(u) - 96)
    object_id = object_id.replace('-','')[:250]
    return object_id

telegram_bot_sendtext('starts bbc')

try:
    url1 = 'https://www.bbc.com/russian'
    page = requests.get(url1)
    page.status_code
    soup = BeautifulSoup(page.text, 'html.parser')
    url_statyi = soup.find_all(class_ = 'bbc-1fxtbkn evnt13t0')
    urls = []
    for i in range(len(url_statyi)):
        url1 = 'https://www.bbc.com' + url_statyi[i].get('href')
        if not re.search('mhttp', url1):
            urls.append(url1)
        q = 0
        
    for i in range(len(urls)):
        URL = urls[i]
        url = urls[i][0:]
        object_id = converter(url)
        page = requests.get(URL)
        page.status_code
        soup =  BeautifulSoup(page.text, 'html.parser')
        
        date = soup.find('time', {'': ''}).get('datetime')
        if date is not None and re.search(xx, date):
            
            try:
                img = soup.find('img', {'class': 'bbc-fat2bc e1enwo3v0'}).get('src')
            except:
                img = None
                
            time = soup.find('time', {'': ''}).get_text()
            if re.search('час назад', time):
                hour = 1
                past_time = current_time - timedelta(hours=hour)
                past_time_str = past_time.strftime('%H:%M:%S') + '.0'
                
            elif re.search('минут', time):
                ho = re.findall('\d+', time)
                minut = int(ho[0])
                past_time = current_time - timedelta(minutes=minut)
                past_time_str = past_time.strftime('%H:%M:%S') + '.0'
                
            else:
                ho = re.findall('\d+', time)
                hour = int(ho[0])
                past_time = current_time - timedelta(hours=hour)
                past_time_str = past_time.strftime('%H:%M:%S') + '.0'
            datetime = date + ' '+ past_time_str
            
            try:
                t = soup.find('main', {'role': 'main'})
                text = soup.find('h1', {'class': 'bbc-7ota8y e1yj3cbb0'}).text + "\n"
                t1 = t.find_all('p', {'class': 'bbc-bm53ic e1cc2ql70'})
                for t2 in t1:
                    text = text + t2.text

            except:
                t = soup.find('main', {'role': 'main'})
                text = soup.find('strong', {'class': 'e8stly50 bbc-1luz917 e14hemmw1'}).text + "\n"
                t1 = t.find_all('p', {'class': 'bbc-bm53ic e1cc2ql70'})
                for t2 in t1:
                    text = text + t2.text
                    
            connection_text = """DATABASE=PRODDB;HOSTNAME=192.168.252.11;PORT=50000;
            PROTOCOL=TCPIP;UID=db2inst1;PWD=Qjuehnghj1;"""
            sql_1_test = "SELECT url_channel FROM TL_MEDIA_DATA_NEWS_TEST"
            con = db.connect(connection_text, "", "")
            cursor = con.cursor()
            cursor.execute(sql_1_test)
            t = cursor.fetchall()
            con.commit()
            cursor.close()
            con.close()

            t0 = []
            for y in range(len(t)):
                t0.append(t[y][0])

            if not URL in t0:
                connection_text = """DATABASE=PRODDB;HOSTNAME=192.168.252.11;
                PORT=50000;PROTOCOL=TCPIP;UID=db2inst1;PWD=Qjuehnghj1;"""
                con = db.connect(connection_text, "", "")
                cursor = con.cursor()
                insert_query = """insert into TL_MEDIA_DATA_NEWS_TEST (object_id, published_date, channel_id, 
                likes, comments, views, reposts, caption, text, url_attachment, url_channel, source_id) 
                values (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)"""
                cursor.execute(insert_query, (object_id, datetime, None, None, None, None, None, None, text, img, URL, 701))
                con.commit()
                cursor.close()
                con.close()
                q = q + 1
        #"Парсинг riafan.ru прошел успешно.(time.kz)" 
    send_text = "Парсинг сайта bbc.com успешно прошло, добавлено " + str(q) + " публикации. Благодарю."
    telegram_bot_sendtext(send_text)   #Сообщение отправлено в телеграмм бот
    print(send_text)
except (Exception, psycopg2.Error) as error:
    send_text = "Ошибка при парсинге bbc.com " + "\n" + str(error)
    telegram_bot_sendtext(send_text)   #Сообщение отправлено в телеграмм бот
    print(send_text, z)

Парсинг сайта bbc.com успешно прошло, добавлено 0 публикации. Благодарю.


In [126]:
import ibm_db_dbi as db
import requests
import re
from bs4 import BeautifulSoup
import urllib.request
import urllib

#Select data from table here
connection_text = """DATABASE=PRODDB;HOSTNAME=192.168.252.11;PORT=50000;
    PROTOCOL=TCPIP;UID=db2inst1;PWD=Qjuehnghj1;"""
con = db.connect(connection_text, "", "")
cursor = con.cursor()
cursor.execute("""
SELECT published_date
FROM TL_MEDIA_DATA_NEWS_TEST
WHERE source_id = '701'
ORDER BY published_date
""")
result = cursor.fetchall()
con.commit()
cursor.close()
con.close()
for x in result:
    print(x)   

(datetime.datetime(2021, 7, 12, 0, 25, 13),)
(datetime.datetime(2021, 7, 12, 0, 25, 13),)
(datetime.datetime(2021, 7, 12, 11, 32, 13),)
(datetime.datetime(2021, 7, 12, 12, 4, 13),)
(datetime.datetime(2021, 7, 12, 14, 0, 10),)


In [124]:
import urllib.request
from bs4 import BeautifulSoup
import re
import requests
import datetime
from datetime import date, timedelta, datetime

date_while = date.today()
xx = date_while.strftime('%Y-%m-%d')
current_time = datetime.now()

page = urllib.request.urlopen("https://www.bbc.com/russian")
soup = BeautifulSoup(page)

h = soup.find_all('a', {'class': 'bbc-1fxtbkn evnt13t0'})
for h1 in h:
    href = h1.get('href')
    
    if not re.search('http', href):
        post = "https://www.bbc.com" + href
        page = urllib.request.urlopen(post)
        soup = BeautifulSoup(page)
        
        date = soup.find('time', {'': ''}).get('datetime')
        if date is not None and re.search(xx, date):
            
            try:
                img = soup.find('img', {'class': 'bbc-fat2bc e1enwo3v0'}).get('src')
            except:
                img = None
                
            time = soup.find('time', {'': ''}).get_text()
            
            if re.search('час назад', time):
                hour = 1
                past_time = current_time - timedelta(hours=hour)
                past_time_str = past_time.strftime('%H:%M:%S') + '.0'
                
            elif re.search('минут', time):
                ho = re.findall('\d+', time)
                minut = int(ho[0])
                past_time = current_time - timedelta(minutes=minut)
                past_time_str = past_time.strftime('%H:%M:%S') + '.0'
                
            else:
                ho = re.findall('\d+', time)
                hour = int(ho[0])
                past_time = current_time - timedelta(hours=hour)
                past_time_str = past_time.strftime('%H:%M:%S') + '.0'
                
            try:
                t = soup.find('main', {'role': 'main'})
                text = soup.find('h1', {'class': 'bbc-7ota8y e1yj3cbb0'}).text + "\n"
                t1 = t.find_all('p', {'class': 'bbc-bm53ic e1cc2ql70'})
                for t2 in t1:
                    text = text + t2.text

            except:
                t = soup.find('main', {'role': 'main'})
                text = soup.find('strong', {'class': 'e8stly50 bbc-1luz917 e14hemmw1'}).text + "\n"
                t1 = t.find_all('p', {'class': 'bbc-bm53ic e1cc2ql70'})
                for t2 in t1:
                    text = text + t2.text
            print(date, past_time_str + "\n" + text + "\n")

2021-07-12 11:46:10.0
Почему российские матери становятся одиночками и как справляются с бедностью. Три истории
Подавляющее большинство малоимущих семей в России - семьи с несовершеннолетними детьми. В особенно тяжелом положении оказались неполные семьи: бедными в России считается более трети семей, где детей двое и больше, а работающий взрослый при этом один. Би-би-си съездила в гости к трем женщинам из российской глубинки и выяснила, как российские матери уходят от мужей, в одиночку начинают бизнес, не справляются с кредитами и мечтают о недостижимом. "Беру две целые курицы. Грудку - на фарш, из него делаю тефтели с рисом. Каркас - на бульон, а ножки поджариваю. В рыбном магазине можно купить навагу по акции за 99 рублей. Перемалываю в мясорубке - получаются рыбные котлеты. Есть у нас магазин "Ярче" - там крупы дешевле. Покупаю гречку, рис, лапшу, масло, сахар. В "Мария Ра" кофе бывает по акции, 95 рублей. А я кофеман, я без кофе не могу". Так Ольга из Барнаула описывает рацион своей

2021-07-12 04:46:10.0
На Кубе вспыхнули антиправительственные протесты. Последний раз такое было 30 лет назад
В кубинской столице Гаване и ряде других городов прошли многотысячные акции протеста против коммунистического правительства страны.Участники акций требуют свободы, демократии и более решительных действий властей в борьбе с пандемией коронавируса. Как сообщает агентство Франс пресс, среди лозунгов демонстрантов - "долой диктатуру" и "мы не боимся".За последние сутки в стране было зарегистрировано рекордное число заболевших. Люди требуют от властей ускорить темпы вакцинации.На прошлой неделе на Кубе была одобрена вакцина собственного производства Abdala 2. Это первая вакцина, созданная в странах Латинской Америки и Карибского бассейна. Однако протестующие говорят, что дело не только в пандемии. Они возмущены растущим дефицитом электроэнергии и продовольствия. Антиправительственные демонстрации - большая редкость для Кубы. Последние подобные выступления состоялись почти 30 лет наз

In [87]:
import datetime
import re

now = datetime.datetime.now()
time = now.strftime("%H:%M:%S")

d = re.findall('\d+', time)
time1 = d[0] + ':' + d[1] + ':' + d[2]

tlist = [time1]

hour = 'час назад'
if re.search('час', hour):
    tlist = [time1, '3:00:00']
    mysum = datetime.timedelta()
    for i in tlist:
        (h, m, s) = i.split(':')
        d = datetime.timedelta(hours=int(h), minutes=int(m), seconds=int(s))
        mysum += d
    print(time, str(mysum))
else:
    print('else')

11:51:25 14:51:25


In [127]:
from datetime import datetime
from datetime import timedelta

# Get current time in local timezone
current_time = datetime.now()
current_time_str = current_time.strftime('%H:%M:%S')
print('Current Time: ', current_time_str)

n = 2

# Subtract 2 hours from datetime object containing current time
past_time = current_time - timedelta(hours=n)

# Convert datetime object to string in specific format 
past_time_str = past_time.strftime('%H:%M:%S')
print('Past Time: ', past_time_str)

Current Time:  16:53:51
Past Time:  14:53:51


In [147]:
import urllib.request
from bs4 import BeautifulSoup
import requests
import os
def main():
    page = urllib.request.urlopen("https://www.cbr.ru/")
    soup = BeautifulSoup(page)
    h = soup.find('div', {'class': 'col-md-2 col-xs-9 _right mono-num'}).text
    return send_message(h)
def send_message(message):
    title = 'Dollar: '
    os.system('notify-send "{}" "{}"'.format(title, message))
main()